In [1]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from linearmodels import PanelOLS
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import ResidualsPlot
from scipy import stats

In [2]:
joint = pd.read_excel('joint_snap.xlsx')
regression = pd.read_excel('AfterRegression.xlsx')
mom_factors = pd.read_excel('Europe_MOM_Factor.xlsx')
three_factors = pd.read_excel('Europe_3_Factors.xlsx')

In [3]:
three_factors = three_factors.rename(columns={three_factors.columns[0]: "Month" })
mom_factors = mom_factors.rename(columns={mom_factors.columns[0]: "Month" })
regression = regression[['FundId','Month','Fund Return']]

three_factors['Month'] = pd.to_datetime(three_factors['Month'])
mom_factors['Month'] = pd.to_datetime(mom_factors['Month'])
joint['Month'] = pd.to_datetime(joint['Month'])
regression['Month'] = pd.to_datetime(regression['Month'])

joint_new = joint.join(mom_factors.set_index('Month'), on='Month')
joint = joint_new.join(three_factors.set_index('Month'), on='Month')

joint_new = pd.merge(joint, regression, on=['FundId', 'Month'])

In [7]:
left = joint_new['Fund Return'] - joint_new['RF']
joint_new['r_it'] = left

In [11]:
#catagorize the month - 2012/01 to 2019/08, totoally 92 months
month = pd.Categorical(joint_new.Month)

#use the 'Month' to divide the dataset, the index is Fund ID
df = joint_new.set_index(['FundId', 'Month'])
df['Month'] = month

In [14]:
#Mkt_RF
exog_vars = ['Mkt-RF']
exog = sm.add_constant(df[exog_vars])

mod = PanelOLS(df.r_it, exog)

res = mod.fit()

In [17]:
#three factor
exog_vars_3 = ['Mkt-RF','SMB','HML']
exog_3 = sm.add_constant(df[exog_vars_3])

mod_3 = PanelOLS(df.r_it, exog_3)

res_3 = mod_3.fit()

In [19]:
#four factor
exog_vars_4 = ['Mkt-RF','SMB','HML','WML']
exog_4 = sm.add_constant(df[exog_vars_4])

mod_4 = PanelOLS(df.r_it, exog_4)

res_4 = mod_4.fit()

In [20]:
print(res_3)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   r_it   R-squared:                        0.4575
Estimator:                   PanelOLS   R-squared (Between):              0.0000
No. Observations:              301392   R-squared (Within):               0.4615
Date:                Sun, Apr 12 2020   R-squared (Overall):              0.4575
Time:                        09:31:13   Log-likelihood                -7.404e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                   8.473e+04
Entities:                        3276   P-value                           0.0000
Avg Obs:                       92.000   Distribution:                F(3,301388)
Min Obs:                       92.000                                           
Max Obs:                       92.000   F-statistic (robust):          8.473e+04
                            

In [21]:
print(res_4)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   r_it   R-squared:                        0.4581
Estimator:                   PanelOLS   R-squared (Between):              0.0000
No. Observations:              301392   R-squared (Within):               0.4621
Date:                Sun, Apr 12 2020   R-squared (Overall):              0.4581
Time:                        09:32:16   Log-likelihood                -7.402e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                   6.369e+04
Entities:                        3276   P-value                           0.0000
Avg Obs:                       92.000   Distribution:                F(4,301387)
Min Obs:                       92.000                                           
Max Obs:                       92.000   F-statistic (robust):          6.369e+04
                            